# Project 2

You have been approached by The Director of Yemisi Shyllon Museum of Art (YSMA) to build a computer vision program with a graphical user interface (GUI) that would ask for a username and password from visitors to log in and view and perform image enhancements on the art collection at the museum. Given the knowledge of the collection categories from the previous project.

Hint: Use the Python Tinker library to build your GUI


In [1]:
import tkinter as tk
from tkinter import messagebox

In [2]:
# Function to validate login credentials
def validate_login():
    username = username_entry.get()
    password = password_entry.get()
    
    # Predefined username and password for demonstration
    if username == "admin" and password == "password":
        messagebox.showinfo("Login Successful", "Welcome to YSMA!")
        # Code to display options for viewing and enhancing images
    else:
        messagebox.showerror("Login Failed", "Invalid username or password")


In [3]:
# Create the main application window
root = tk.Tk()
root.title("YSMA Art Collection Viewer")

# Create and place login widgets
username_label = tk.Label(root, text="Username:")
username_label.grid(row=0, column=0, padx=10, pady=5, sticky="e")
username_entry = tk.Entry(root)
username_entry.grid(row=0, column=1, padx=10, pady=5)

password_label = tk.Label(root, text="Password:")
password_label.grid(row=1, column=0, padx=10, pady=5, sticky="e")
password_entry = tk.Entry(root, show="*")
password_entry.grid(row=1, column=1, padx=10, pady=5)

login_button = tk.Button(root, text="Login", command=validate_login)
login_button.grid(row=2, column=1, padx=10, pady=5)

# Start the Tkinter event loop
root.mainloop()